In [9]:
import cvxpy as cp
import numpy as np
import itertools
from itertools import product as iters

In [111]:
# Problem 1 - Solving for Bell functionals
n = 2 # 2 should give the CHSH parameter 
# Variables
sizeOfP = 2 * 2 * n * n 
P = cp.Variable(sizeOfP)

# Constraints
constraints = [P >= 0, cp.sum(P) == n*2]
for a, b in iters([-1, 1], repeat = 2):
    constraints += [cp.sum(findCombinationsOf_ab(P, a, b, n)) == 1]
for xT in range(1, n + 1):
    for yT in range(1, n + 1):
        constraints += [cp.sum(findCombinationsOf_xy(P, xT, yT, n)) == 2/n]

# Function to be maximized
prob = cp.Problem(cp.Maximize(getB(P, n)), constraints)
print("At n=2")
useAllSolvers(prob, False)
for i in range(3, 7):
    print("At n=", i, " ")
    sizeOfP = 2 * 2 * i * i 
    P = cp.Variable(sizeOfP)
    prob = cp.Problem(cp.Maximize(getB(P, i)), constraints)
    useAllSolvers(prob, True)

At n=2
optimal value with OSQP: 2.0
optimal value with ECOS: 1.9999999999775775
optimal value with SCS: 2.0000098849124495
optimal value with SciPy/HiGHS: 2.0
At n= 3  
optimal value with OSQP: inf
optimal value with ECOS: inf
optimal value with SCS: inf
optimal value with SciPy/HiGHS: inf
At n= 4  
optimal value with OSQP: inf
optimal value with ECOS: inf
optimal value with SCS: inf
optimal value with SciPy/HiGHS: inf
At n= 5  
optimal value with OSQP: inf
optimal value with ECOS: inf
optimal value with SCS: inf
optimal value with SciPy/HiGHS: inf
At n= 6  
optimal value with OSQP: inf
optimal value with ECOS: inf
optimal value with SCS: inf
optimal value with SciPy/HiGHS: inf


**The above approach failed, trying a different**

In [119]:
def getGamma(P, n):
    zeroMat = np.zeros((n, n))
    return np.block([[zeroMat, P.T], [P, zeroMat]])

def getP(n):
    P = np.zeros((n, n))
    # Diagonal ones
    for i in range(0, n):
        P[i, i] = 1
    for i in range(0, n - 1):
        P[i + 1, i] = 1
    P[0, n - 1] = -1
    print(P)
    return P

# Now for all n values...
for n in range(2, 7):
    P = getP(n)
    gamma = getGamma(P, n)

    # Set Positive semi-definite constrain with the one built into cvxpy
    X = cp.Variable((2 * n, 2* n), PSD=True)

    #  Build diagonal constrains
    constraints =[]
    for i in range(0, 2 * n):
        constraints += [X[i, i] == 1]
    
    obj = cp.Maximize(cp.trace(gamma @ X) / 2)
    prob = cp.Problem(obj, constraints)
    print("At n = ", n, ": Max val = ", prob.solve() )

[[ 1. -1.]
 [ 1.  1.]]
At n =  2 : Max val =  2.828427124639426
[[ 1.  0. -1.]
 [ 1.  1.  0.]
 [ 0.  1.  1.]]
At n =  3 : Max val =  5.196152427563782
[[ 1.  0.  0. -1.]
 [ 1.  1.  0.  0.]
 [ 0.  1.  1.  0.]
 [ 0.  0.  1.  1.]]
At n =  4 : Max val =  7.391037008387394
[[ 1.  0.  0.  0. -1.]
 [ 1.  1.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.]
 [ 0.  0.  1.  1.  0.]
 [ 0.  0.  0.  1.  1.]]
At n =  5 : Max val =  9.510593333353807
[[ 1.  0.  0.  0.  0. -1.]
 [ 1.  1.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.]
 [ 0.  0.  0.  1.  1.  0.]
 [ 0.  0.  0.  0.  1.  1.]]
At n =  6 : Max val =  11.591107813376762


# Problem 2

The condition for information causality is given by
$$
    \sum_y I(b:X_y) \le m = 1
$$
where 
$$
    I(b:X_y) = H(b) - H(X_y) - H(b, X_y)
$$
with H(b, X_y) is the joint distribution and $I(b:X_y) = I(b:X_0) + I(b:X_1)$

Entropy $H(x)$ is given by $H(x) = -P(x)log_2 (P(x))$ which then for
$$
    H(b, X_y) = - \sum_{b, y} P(b, X_y) log_2(P(b, X_y))
$$

Solving this with sympy

In [6]:
import sympy as sy
from sympy.abc import a,b,x,y,l
from sympy import DiracDelta

In [32]:
x0 = sy.Symbol("x_0")
x1 = sy.Symbol("x_1")
l = sy.Symbol("λ")

In [33]:
# Entropy
def H(x):
    return -x*sy.log(x, 2)

# A uniform distribution
def P_uni():
    return 1/2

# An isotropic distribution
def P_iso(a, b, x, y, l):
    p1 = DiracDelta(((a + b) % 2) - x * y)
    p0 = DiracDelta(((a + b + 1) % 2) - x * y)
    return ((l * p1 + (1 - l) * p0) / 2).subs(DiracDelta(0), 1)

HX_0 = np.sum(np.array([H(P_uni()) for n in range(0, 2)])).evalf()


In [34]:
HX_0 # Sanity check

1.00000000000000

In [35]:
HX_1 = HX_0
HX_01 = HX_0 + HX_1

# For y = 0
y = 0 
P_00 = 0
P_10 = 0
for a, b, X_0 in iters([0, 1], repeat = 3):
    if a ^ b ^ X_0 == 0:
        P_00 += P_iso(a, b, X_0, y, 1) / 2
    elif a ^ b ^ X_0 == 1:
        P_10 += P_iso(a, b, X_0, y, 1) / 2


In [36]:
# Check results
print(P_00, P_10)

1/2 1/2


In [37]:
# Now for y = 1
# For y = 0
y = 0 
P_01 = 0
P_11 = 0
for a, b, X_0 in iters([0, 1], repeat = 3):
    if a ^ b ^ X_0 == 0:
        P_01 += P_iso(a, b, X_0, y, l) / 2
    elif a ^ b ^ X_0 == 1:
        P_11 += P_iso(a, b, X_0, y, l) / 2

In [38]:
# Check results
print(P_10, P_11)

1/2 1/2


In [39]:
Hb = sy.Matrix([H(P_00), H(P_01), H(P_10), H(P_11)])

In [40]:
print(Hb)

Matrix([[1/2], [1/2], [1/2], [1/2]])


In [41]:
# Now the joint distribution for y = 0
y = 0
jointP_0 = []
for bT, X_0 in iters([0, 1], repeat=2):
    t = 0
    for a, b in iters([0, 1], repeat=2):
        if a ^ b ^ X_0 == bT:
            t += P_iso(a, b, X_0, y, l) / 2
    jointP_0.append(t)

In [45]:
# And for y = 1
y = 1
jointP_1 = []
for bT, X_1 in iters([0, 1], repeat=2):
    t = 0
    for a, b in iters([0, 1], repeat=2):
        if a ^ b ^ X_1 == bT:
            t += P_iso(a, b, X_1, y, l) / 2
    jointP_1.append(t)

In [49]:
sy.Matrix(jointP_0) 

Matrix([
[      λ/2],
[1/2 - λ/2],
[1/2 - λ/2],
[      λ/2]])

In [50]:
sy.Matrix(jointP_1)

Matrix([
[      λ/2],
[      λ/2],
[1/2 - λ/2],
[1/2 - λ/2]])

In [51]:
# Calculate the joint entropies
H_j0 = 0
H_j1 = 0
for i in range(0, 4):
    H_j0 += H(jointP_0[i])
    H_j1 += H(jointP_1[i])

In [52]:
H_j0

-λ*log(λ/2)/log(2) + 2*(λ/2 - 1/2)*log(1/2 - λ/2)/log(2)

In [53]:
H_j1

-λ*log(λ/2)/log(2) + 2*(λ/2 - 1/2)*log(1/2 - λ/2)/log(2)

Rewriting this as $H_{joint} = 1 - \lambda log_2 (\lambda) + (\lambda - 1) log_2 (1- \lambda)$ leads to
$$
    I(b:X_i) = 1- \lambda log_2(\lambda) + (1-\lambda)log_2(1-\lambda)
$$
Which, after evaluating using the constraint inequality from the beginning yields:
$$
    0.110028 \lt \lambda \lt 0.889972
$$
This is the range in which the inequality holds true!

Now, given an isotropic box:
$$ 
CHSH(P) = \mu
$$
with $\lambda = \mu / 8 + 1/2$. This gives $\mu = 8 * \lambda - 4$
which for the maximum violation 0.889972 yields: 

In [56]:
8 * 0.889972 - 4

3.119776

*Exceeds Tsirelsons bound without violating Information Causality!*